In [1]:
import pandas as pd
import numpy as np
# 모든 열이 출력되도록 설정
pd.set_option('display.max_columns', None)

df = pd.read_csv("../data/processed/db_network_data_table.csv")
df = df.drop(columns=['id', 'source'])
# 12.0s

In [2]:
# 상관계수 계산
correlation_matrix = df.corr()

# 상삼각형 인덱스만 가져오기 (k=1: 자기 자신을 제외한 값들만 선택)
upper_triangle = np.triu_indices_from(correlation_matrix, k=1)

# 상관계수 절대값이 0.8 이상인 변수들만 추출
threshold = 0.8
high_corr_pairs = []  # 상관계수 0.8 이상인 변수 쌍을 저장할 리스트

# 상관계수가 0.8 이상인 변수 쌍을 찾아서 리스트에 추가
for i, j in zip(upper_triangle[0], upper_triangle[1]):
    corr_value = correlation_matrix.iloc[i, j]
    
    # 상관계수가 절대값으로 0.8 이상이면 변수 쌍을 리스트에 추가
    if abs(corr_value) >= threshold:
        high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], corr_value))

# 상관관계가 높은 변수 쌍 출력
print(f"상관계수가 {threshold} 이상인 변수 쌍: {len(high_corr_pairs)}")
for pair in high_corr_pairs:
    print(f"{pair[0]} - {pair[1]}: {pair[2]:.2f}")

# 다중공선성 파악을 위한 변수 선택
if high_corr_pairs:
    # 상관계수가 0.8 이상인 변수들만 추출하여 리스트로 만듦
    correlated_columns = list({x[0] for x in high_corr_pairs} | {x[1] for x in high_corr_pairs})
    print("\n다중공선성 가능성이 높은 변수들:")
    print(correlated_columns)
else:
    print("다중공선성 문제는 발견되지 않았습니다.")

# 26.2s

상관계수가 0.8 이상인 변수 쌍: 101
Flow Duration - Fwd IAT Total: 1.00
Flow Duration - Bwd IAT Total: 0.82
Total Fwd Packets - Total Backward Packets: 1.00
Total Fwd Packets - Total Length of Bwd Packets: 1.00
Total Fwd Packets - Subflow Fwd Packets: 1.00
Total Fwd Packets - Subflow Bwd Packets: 1.00
Total Fwd Packets - Subflow Bwd Bytes: 1.00
Total Fwd Packets - act_data_pkt_fwd: 0.89
Total Backward Packets - Total Length of Bwd Packets: 0.99
Total Backward Packets - Subflow Fwd Packets: 1.00
Total Backward Packets - Subflow Bwd Packets: 1.00
Total Backward Packets - Subflow Bwd Bytes: 0.99
Total Backward Packets - act_data_pkt_fwd: 0.88
Total Length of Fwd Packets - Subflow Fwd Bytes: 1.00
Total Length of Bwd Packets - Subflow Fwd Packets: 1.00
Total Length of Bwd Packets - Subflow Bwd Packets: 0.99
Total Length of Bwd Packets - Subflow Bwd Bytes: 1.00
Total Length of Bwd Packets - act_data_pkt_fwd: 0.88
Fwd Packet Length Max - Fwd Packet Length Mean: 0.89
Fwd Packet Length Max - Fwd Packet Len

In [3]:
# 단 하나의 값으로만 이루어진 칼럼 찾기
single_value_columns = [col for col in df.columns if df[col].nunique() == 1]
df_cleaned = df.drop(columns=single_value_columns)

# 결과 출력
print("제거된 단 하나의 값으로만 이루어진 칼럼:")
print(single_value_columns)

# 2.5s

제거된 단 하나의 값으로만 이루어진 칼럼:
['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']


In [4]:
# 각 칼럼의 값을 tuple로 변환하여 비교
grouped_columns = {}

for col in df_cleaned.columns:
    col_values = tuple(df_cleaned[col].values)  # 각 칼럼의 값을 tuple로 변환
    if col_values in grouped_columns:  # 동일한 값이 이미 존재하는지 확인
        grouped_columns[col_values].append(col)  # 그룹에 추가
    else:
        grouped_columns[col_values] = [col]  # 새로운 그룹 생성

# 그룹화된 칼럼 중 2개 이상 포함된 것만 추출
grouped_result = [group for group in grouped_columns.values() if len(group) > 1]

# 결과 출력
print("그룹화된 칼럼들:")
for group in grouped_result:
    print(group)

# 14.4s

그룹화된 칼럼들:
['Total Fwd Packets', 'Subflow Fwd Packets']
['Total Backward Packets', 'Subflow Bwd Packets']
['Fwd Packet Length Mean', 'Avg Fwd Segment Size']
['Bwd Packet Length Mean', 'Avg Bwd Segment Size']
['Fwd PSH Flags', 'SYN Flag Count']
['Fwd URG Flags', 'CWE Flag Count']
['Fwd Header Length', 'Fwd Header Length.1']


In [5]:
# 그룹화된 칼럼에서 첫 번째 칼럼을 제외한 나머지 칼럼들을 columns_to_drop에 추가
columns_to_drop = [col for group in grouped_result for col in group[1:]]

# df_cleaned에서 동일한 칼럼 삭제
df_cleaned = df_cleaned.drop(columns=columns_to_drop)

# 결과 출력
print("제거된 중복 칼럼:")
print(columns_to_drop)

# 2.6s

제거된 중복 칼럼:
['Subflow Fwd Packets', 'Subflow Bwd Packets', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'SYN Flag Count', 'CWE Flag Count', 'Fwd Header Length.1']


In [6]:
# 상관계수 행렬 계산
correlation_matrix = df_cleaned.corr()

# 상삼각형 인덱스만 가져오기 (k=1: 자기 자신을 제외한 값들만 선택)
upper_triangle = np.triu_indices_from(correlation_matrix, k=1)

# 상관계수 절대값이 0.9 이상인 변수들만 추출
threshold = 0.9
columns_to_drop = []  # 삭제할 변수들을 저장할 리스트
high_corr_pairs = []  # 상관계수 0.9 이상인 변수 쌍을 저장할 리스트

# 상관계수가 0.9 이상인 변수 쌍을 찾아서 리스트에 추가
for i, j in zip(upper_triangle[0], upper_triangle[1]):
    corr_value = correlation_matrix.iloc[i, j]
    
    # 상관계수가 절대값으로 0.9 이상이면
    if abs(corr_value) >= threshold:
        columns_to_drop.append(correlation_matrix.columns[j])  # 두 번째 변수만 추가 (첫 번째 변수는 남김)
        high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], corr_value))  # 변수 쌍과 상관계수 저장

# 중복 제거 (세 번째 변수는 이미 두 번 이상 리스트에 포함될 수 있음)
columns_to_drop = list(set(columns_to_drop))

# 결과 출력
print(f"상관계수 0.9 이상인 변수 쌍 개수: {len(high_corr_pairs)}")
print("상관계수가 0.9 이상인 변수 쌍:")
for pair in high_corr_pairs:
    print(f"{pair[0]} - {pair[1]}: {pair[2]:.2f}")

# 삭제할 칼럼들 출력
print(f"\n삭제 고려 할 칼럼 개수: {len(columns_to_drop)}")
print("삭제 고려 할  칼럼들:", columns_to_drop)

# 17.3s

상관계수 0.9 이상인 변수 쌍 개수: 51
상관계수가 0.9 이상인 변수 쌍:
Flow Duration - Fwd IAT Total: 1.00
Total Fwd Packets - Total Backward Packets: 1.00
Total Fwd Packets - Total Length of Bwd Packets: 1.00
Total Fwd Packets - Subflow Bwd Bytes: 1.00
Total Backward Packets - Total Length of Bwd Packets: 0.99
Total Backward Packets - Subflow Bwd Bytes: 0.99
Total Length of Fwd Packets - Subflow Fwd Bytes: 1.00
Total Length of Bwd Packets - Subflow Bwd Bytes: 1.00
Fwd Packet Length Max - Fwd Packet Length Std: 0.97
Bwd Packet Length Max - Bwd Packet Length Mean: 0.96
Bwd Packet Length Max - Bwd Packet Length Std: 0.98
Bwd Packet Length Max - Max Packet Length: 0.95
Bwd Packet Length Max - Packet Length Std: 0.94
Bwd Packet Length Mean - Bwd Packet Length Std: 0.94
Bwd Packet Length Mean - Max Packet Length: 0.91
Bwd Packet Length Mean - Packet Length Mean: 0.92
Bwd Packet Length Mean - Packet Length Std: 0.92
Bwd Packet Length Mean - Average Packet Size: 0.92
Bwd Packet Length Std - Max Packet Length: 0.93
Bwd

In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# 상수 항 추가 (회귀 분석을 위한 절차)
X = add_constant(df_cleaned)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# VIF가 5 이상인 변수
high_vif_columns = vif_data[vif_data["VIF"] > 5]["Variable"]

print(high_vif_columns)

# 6m 40.2s

0                           const
2                   Flow Duration
3               Total Fwd Packets
4          Total Backward Packets
5     Total Length of Fwd Packets
6     Total Length of Bwd Packets
7           Fwd Packet Length Max
9          Fwd Packet Length Mean
10          Fwd Packet Length Std
11          Bwd Packet Length Max
13         Bwd Packet Length Mean
14          Bwd Packet Length Std
16                 Flow Packets/s
17                  Flow IAT Mean
18                   Flow IAT Std
19                   Flow IAT Max
20                   Flow IAT Min
21                  Fwd IAT Total
22                   Fwd IAT Mean
23                    Fwd IAT Std
24                    Fwd IAT Max
25                    Fwd IAT Min
26                  Bwd IAT Total
27                   Bwd IAT Mean
28                    Bwd IAT Std
29                    Bwd IAT Max
30                    Bwd IAT Min
35                  Fwd Packets/s
36                  Bwd Packets/s
38            

In [8]:
from sklearn.decomposition import PCA

X = df_cleaned.drop(columns=['Label'])

# 2개의 주성분을 사용하는 PCA 모델 생성
pca_2_model = PCA(n_components=2)
pca_2_model.fit(X)
pca_2 = pca_2_model.fit_transform(X)

# 3개의 주성분을 사용하는 PCA 모델 생성
pca_3_model = PCA(n_components=3)
pca_3_model.fit(X)
pca_3 = pca_3_model.fit_transform(X)

# 28.1s

In [9]:
# 로딩 스코어 (2개 주성분)
loading_scores_2 = pd.DataFrame(pca_2_model.components_.T, columns=['PC1', 'PC2'], index=X.columns)
print("로딩 스코어 (2개 주성분):")
print(loading_scores_2)

# 로딩 스코어를 (3개 주성분)
loading_scores_3 = pd.DataFrame(pca_3_model.components_.T, columns=['PC1', 'PC2', 'PC3'], index=X.columns)
print("로딩 스코어 (3개 주성분):")
print(loading_scores_3)

# 0.0s

로딩 스코어 (2개 주성분):
                                      PC1           PC2
Destination Port            -3.447337e-05 -2.082575e-05
Flow Duration                4.360958e-01  3.093283e-01
Total Fwd Packets            1.249018e-07  6.625790e-07
Total Backward Packets       1.554767e-07  8.598833e-07
Total Length of Fwd Packets  6.831999e-06  2.110705e-05
...                                   ...           ...
Active Min                   9.753808e-04 -2.254467e-04
Idle Mean                    3.009451e-01 -2.808334e-01
Idle Std                     1.478669e-02 -4.405093e-04
Idle Max                     3.120336e-01 -2.800004e-01
Idle Min                     2.896461e-01 -2.850908e-01

[63 rows x 2 columns]
로딩 스코어 (3개 주성분):
                                      PC1           PC2           PC3
Destination Port            -3.447337e-05 -2.082575e-05  4.293370e-05
Flow Duration                4.360958e-01  3.093283e-01  7.300873e-03
Total Fwd Packets            1.249018e-07  6.625790e-07  3.46

In [10]:
least_contributors_pc1 = loading_scores_2['PC1'].abs().nsmallest(int(len(loading_scores_2) * 0.2)).index
least_contributors_pc2 = loading_scores_2['PC2'].abs().nsmallest(int(len(loading_scores_2) * 0.2)).index

# PC1과 PC2에서 공통으로 낮은 기여도를 가지는 칼럼
least_selected_columns = list(set(least_contributors_pc1).intersection(set(least_contributors_pc2)))
print("제거 가능성이 높은 칼럼들:")
print(least_selected_columns)

# 로딩 스코어 출력
loading_scores_least_selected = loading_scores_2.loc[least_selected_columns]
print("선택된 칼럼들의 로딩 스코어:")
print(loading_scores_least_selected)

# 0.0s

제거 가능성이 높은 칼럼들:
['FIN Flag Count', 'RST Flag Count', 'Down/Up Ratio', 'URG Flag Count', 'ACK Flag Count', 'Min Packet Length', 'PSH Flag Count', 'Fwd PSH Flags', 'Fwd Packet Length Min', 'ECE Flag Count', 'Fwd URG Flags']
선택된 칼럼들의 로딩 스코어:
                                PC1           PC2
FIN Flag Count         7.201223e-10 -5.538335e-10
RST Flag Count         1.019152e-12 -2.102362e-13
Down/Up Ratio         -1.105433e-09 -2.700758e-10
URG Flag Count        -3.718065e-10 -2.546255e-10
ACK Flag Count         4.209147e-10 -3.717217e-09
Min Packet Length     -6.821577e-08 -5.558300e-08
PSH Flag Count         3.355196e-10  5.425472e-09
Fwd PSH Flags         -3.255457e-11  8.836659e-11
Fwd Packet Length Min -7.659208e-08 -6.348257e-08
ECE Flag Count         1.012997e-12 -2.131966e-13
Fwd URG Flags         -6.469133e-13 -3.104161e-13


In [11]:
least_contributors_pc1 = loading_scores_3['PC1'].abs().nsmallest(int(len(loading_scores_3) * 0.2)).index
least_contributors_pc2 = loading_scores_3['PC2'].abs().nsmallest(int(len(loading_scores_3) * 0.2)).index
least_contributors_pc3 = loading_scores_3['PC3'].abs().nsmallest(int(len(loading_scores_3) * 0.2)).index

# PC1과 PC2가 PC3 에서 공통으로 낮은 기여도를 가지는 칼럼
least_selected_columns = list(set(least_contributors_pc1).intersection(set(least_contributors_pc2),set(least_contributors_pc3)))
print("제거 가능성이 높은 칼럼들:")
print(least_selected_columns)

# 로딩 스코어 출력
loading_scores_least_selected = loading_scores_3.loc[least_selected_columns]
print("선택된 칼럼들의 로딩 스코어:")
print(loading_scores_least_selected)

# 0.0s

제거 가능성이 높은 칼럼들:
['FIN Flag Count', 'RST Flag Count', 'Down/Up Ratio', 'URG Flag Count', 'ACK Flag Count', 'Min Packet Length', 'PSH Flag Count', 'Fwd PSH Flags', 'ECE Flag Count', 'Fwd URG Flags']
선택된 칼럼들의 로딩 스코어:
                            PC1           PC2           PC3
FIN Flag Count     7.201223e-10 -5.538335e-10  1.638732e-11
RST Flag Count     1.019152e-12 -2.102362e-13 -5.001659e-13
Down/Up Ratio     -1.105433e-09 -2.700758e-10 -1.159195e-09
URG Flag Count    -3.718065e-10 -2.546255e-10 -2.160499e-10
ACK Flag Count     4.209147e-10 -3.717217e-09  8.728222e-10
Min Packet Length -6.821577e-08 -5.558300e-08  6.567895e-09
PSH Flag Count     3.355196e-10  5.425472e-09 -4.684635e-10
Fwd PSH Flags     -3.255457e-11  8.836659e-11  1.377659e-09
ECE Flag Count     1.012997e-12 -2.131966e-13 -5.027659e-13
Fwd URG Flags     -6.469133e-13 -3.104161e-13 -3.022112e-13


In [12]:
# PCA 모델 정의, 전체 분산의 99.9%를 유지
pca = PCA(n_components=0.999)
# PCA 적용 및 변환
X_pca_999 = pca.fit_transform(X)

# 사용된 주성분의 수
print("사용된 주성분의 수:", pca.n_components_)

# 각 주성분의 설명된 분산 비율
print("주성분별 설명된 분산 비율:", pca.explained_variance_ratio_)

사용된 주성분의 수: 15
주성분별 설명된 분산 비율: [6.64487576e-01 1.14377839e-01 8.42614753e-02 5.56479602e-02
 4.27153659e-02 2.11825361e-02 7.89138432e-03 3.55127570e-03
 1.29045461e-03 1.25311725e-03 9.23493666e-04 5.39443938e-04
 3.84921571e-04 3.38907426e-04 3.32112384e-04]


In [13]:
pca = PCA(n_components=10)
X_pca_10 = pca.fit_transform(X)

# 사용된 주성분의 수
print("사용된 주성분의 수:", pca.n_components_)

# 각 주성분의 설명된 분산 비율
print("주성분별 설명된 분산 비율:", pca.explained_variance_ratio_)

# 전체 설명된 분산 비율
print("전체 설명된 분산 비율:", pca.explained_variance_ratio_.sum())

사용된 주성분의 수: 10
주성분별 설명된 분산 비율: [0.66448758 0.11437784 0.08426148 0.05564796 0.04271537 0.02118254
 0.00789138 0.00355128 0.00129045 0.00125312]
전체 설명된 분산 비율: 0.9966589847989701
